In [1]:
import mediapipe as mp
import numpy as np
import torch
#import torch_geometric
from PIL import Image
import os
import skimage
import IPython.display
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import clip

from collections import OrderedDict
import torch

from torch_geometric.data import Data
import cv2
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import glob
import os
import math 
from IPython.display import clear_output
import itertools
import seaborn as sb
from tqdm import tqdm


In [2]:
kernel_size= 20
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh

connections = mp_face_mesh.FACEMESH_TESSELATION
edges = []
for connection in connections:
    edge = [connection[0], connection[1]]
    edges.append(edge)
    edge  =[connection[1], connection[0]]
    edges.append(edge)
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()

In [3]:
model, preprocess = clip.load("ViT-B/16")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 149,620,737
Input resolution: 224
Context length: 77
Vocab size: 49408


In [4]:

class EarlyCLIP(nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.conv1 = original_model.visual.conv1
        self.ln_pre = original_model.visual.ln_pre
        self.class_embedding = original_model.visual.class_embedding
        self.positional_embedding = original_model.visual.positional_embedding
        self.transformer = original_model.visual.transformer.resblocks[:2]

    def forward(self, x):
        x = self.conv1(x)
        x = x.reshape(x.shape[0], x.shape[1], -1)
        x = x.permute(0, 2, 1)
        x = torch.cat(
            [self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x],
            dim=1
        )
        x = x + self.positional_embedding.to(x.dtype)
        x = self.ln_pre(x)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        return x.mean(dim=0)
first_two_blocks = EarlyCLIP(model)

In [5]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/oulu/preposess/*/cropped*.jpg")

In [6]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        if os.path.exists( os.path.dirname(path)+'/clip_'+os.path.basename(path)+'.pt'):
            continue
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #print(img.shape)

        landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

        list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
        #print(list_of_points_face)

        list_of_points_face = np.clip(list_of_points_face,0,256)

        list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

        img = np.pad(img, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                 mode='constant', constant_values=0) 


        patches= []

        list_of_points_face= np.round(list_of_points_face).astype(int)

        for (x1, y1) in list_of_points_face:

             p_img = img[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]
             p_img = Image.fromarray(p_img)
             p_img= preprocess(p_img)
             p_img = np.array(p_img)
             patches.append(p_img)

        patches = np.array(patches,dtype =np.float32)
        patches = torch.tensor(patches)
        patches = patches.to("cuda").half() 
        with torch.no_grad():
                features = first_two_blocks(patches).float()
            #np.save(os.path.dirname(path) + "/" + "CLIP_" + os.path.basename(path) + ".npy", image_features.cpu().numpy())        
        #print(features.shape)
        vertices = torch.tensor(np.array(features.cpu()), dtype=torch.float).contiguous()   

        data = Data(x=vertices, edge_index=edges,y =label)
        #torch.save(data, os.path.dirname(path)+'/clip_'+os.path.basename(path)+'.pt')


Processing images: 100%|███████████████████████████████████████████████████████████████████████████| 136334/136334 [6:55:58<00:00,  5.46it/s]
